Notebook purpose:

- Explore whether we can analyse effect of MDB adoption

In [3]:
import sys

import pandas as pd
import s3fs

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [9]:
fp = "s3://3di-project-entropy/entropy_X77.parquet"
df = ha.read_parquet(fp)
hd.inspect(df)

(1,724,896, 31)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
0,51188,2012-02-01,177,4.15,jersey pottery caf sthlr king st,NaN,NaN,NaN,1.0,gu14 9,2010-07-11,NaN,NaN,2011-12-31,178553,2020-06-30 14:04:00,barclaycard,credit card,2014-07-18,2017-10-23,True,-114.349998,NaN,NaN,NaN,dining and drinking,u,201202,3674.744141,38539.005682,2.9614
1,51187,2012-02-01,177,9.99,spotify m1nkeh london 9 99 pound sterling united kingdom,spotify,spend,hobbies,1.0,gu14 9,2010-07-11,NaN,NaN,2011-12-31,178553,2020-06-30 14:04:00,barclaycard,credit card,2014-07-18,2016-04-03,True,-114.349998,spotify,music,media bundle,media bundle,u,201202,3674.744141,38539.005682,2.9614


## Data availability

Overall proportion of txns taking place before user signup.

In [20]:
def pre_signup_share(df):
    return sum(df.date < df.user_registration_date) / len(df)


print("{:.1%} of observations are pre-signup.".format(pre_signup_share(df)))

41.2% of observations are pre-signup.


Distribution of pre-signup txns by user.

In [22]:
df.groupby("user_id").apply(pre_signup_share).describe()

count    598.000000
mean       0.507136
std        0.368577
min        0.000000
25%        0.086278
50%        0.508084
75%        0.885884
max        1.000000
dtype: float64

Looks great. Could select users with certain minimal number of overall txns and with pre-signup share of 25-75 percent or something similar (based on power calculation).